# Object Detection


This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


## Example use

In [ ]:
#@title Imports and function definitions
import tensorflow_hub as hub
import cv2
# For running inference on the TF-Hub module （在TF-HUB Model上可以跑指令）
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.可以在圖片上加上顏色、字體
# PIL(Pillow)是 Python 中著名的影像處理套件
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time. 計算推理時間
import time

# Print Tensorflow version 檢查版本
print(tf.__version__)

# Check available GPU devices. 檢查GPU是否可用
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

In [ ]:
# Display an image inside the notebook
# This is used by download_and_resized_image()
#功能是用來顯示圖片的，且會在download_and_resize_image()被使用
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)#plt.grid 用於配置網格線
  plt.imshow(image)#plt.imshow 顯示圖片


   
#獲取線上或本地端存儲的圖像，調整大小並保存在本地端    
def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
#url_path (string) -- 鏈接到圖像或本地存儲圖像的路徑
#new_width (int) -- 用於調整圖像寬度的像素大小
#new_height (int) -- 用於調整圖像長度的像素大小
#Returns:
    #(string) -- 保存圖像的路徑


  _, filename = tempfile.mkstemp(suffix=".jpg") #創建一個以“.jpg”結尾的暫時檔案 

  response = urlopen(url)# urlopen()打開給定的URL
  image_data = response.read() #response.read()讀取從 URL 中獲取的圖像 
  image_data = BytesIO(image_data) #BytesIO()將圖像數據放入內存緩衝區
  pil_image = Image.open(image_data) # Image.open打開圖片

  # ImageOps.fit()調整圖像大小。如果寬高比不同，將會被裁剪。
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  #  pil_image.convert()轉換為RGB色彩模式
  pil_image_rgb = pil_image.convert("RGB")
  # pill_image_rgb.save()將圖像保存到之前創建的臨時file中
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

##Adds a bounding box to an image.在圖片上加上
def draw_bounding_box_on_image(image,# a cv2 object
                               ymin,#ymin of bounding box.
                               xmin,#xmin: xmin of bounding box
                               ymax,#ymax: ymax of bounding box
                               xmax,#xmax: xmax of bounding box.
                               color,#框框的顏色
                               font,
                               thickness=3,#thickness: line thickness匡線粗度
                               display_str_list=()):
                              #要在框中顯示的字符串列表

  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size#獲取圖像的寬度與高度
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  #繪製box框
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

 #如果添加到邊界框頂部的顯示字符串的總高度超過了圖像的頂部,則將字符串堆疊在邊界框下方而不是上方
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  #每個 display_str 的頂部和底部邊距為 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    #draw.rectangle()繪製文字背景匡
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    #draw.text()繪製文字
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

#draw_boxes():用格式化的分數和標籤名稱在圖像上疊加標籤框
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
# image 圖像作為 numpy array
# boxes: list of detection boxes
# class_names: 每個檢測到的對象的類別
# scores :顯示模型檢測該對象的信心的數字(覺得幾成準確)
# max_boxes：覆蓋在圖像上的最大檢測框（默認為 10）
# min_score:顯示邊界框所需的最低分數
# return image:檢測框和類覆蓋在原始圖像上之後的圖像。
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      #繪製一個邊界框並將類標籤疊加到圖像上
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
# link to the model that we want to use
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
width = 800
height = 600

In [ ]:
def load_img(path):#下載 JPEG 圖像並將其轉換為張量。
  img = tf.io.read_file(path)#讀檔案
  img = tf.image.decode_jpeg(img, channels=3)#轉換為張量
  return img

In [ ]:
#使用物件檢測模型對本機文件進行推測
def run_detector(detector, path):
  img = load_img(path)#從本地文件路徑下載圖像張量

  #在張量前面添加一個批次維度
  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  #使用模型運行推理
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  #將結果保存在字典中
  result = {key:value.numpy() for key,value in result.items()}

  #印出結果
  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  #在圖像上繪製預測框
  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])
  #展示圖像
  display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)

### More images
Perform inference on some additional images with time tracking.


In [ ]:
image_urls = [
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  "https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg",
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg",
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  "https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg",
  
  ]

def detect_img(image_url):#和run_detector()相似
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 740, 480)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",end_time-start_time)

In [ ]:
detect_img(image_urls[0])

In [ ]:
detect_img(image_urls[1])

In [ ]:
detect_img(image_urls[2])

In [ ]:
image_urls_new = ['https://cdn25.img.ria.ru/images/07e4/03/19/1569112614_0:158:3079:1890_600x0_80_0_0_321c57669c44ac781c4105086f65f3f5.jpg',
                  'https://s0.rbk.ru/v6_top_pics/media/img/8/13/755906455134138.jpg',
                  'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Macropus_giganteus_-_Brunkerville.jpg/1200px-Macropus_giganteus_-_Brunkerville.jpg']

In [ ]:
def detect_img_new(image_url):
  start_time = time.time()
  image_path = download_and_resize_image(image_url, 740, 600)
  run_detector(detector, image_path)
  end_time = time.time()
  print("Inference time:",end_time-start_time)

In [ ]:

detect_img_new(image_urls_new[0])

In [ ]:
detect_img_new(image_urls_new[1])

In [ ]:
detect_img_new(image_urls_new[2])